# Grafice raport CDOS

Acest Jupyter Notebook permite generarea unor grafice pe baza răspunsurilor la chestionarul CDOS.

## Import de biblioteci

In [ ]:
from collections.abc import Callable
from pathlib import Path
import re
from textwrap import fill
import string
import unicodedata

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.font_manager import findfont, FontProperties

## Configurare

### Configurare folder de input

Fișierele Excel se vor citi din acest fișier.

In [ ]:
input_directory = Path('.')

### Configurare folder de output

Toate graficele generate se vor salva în acest loc.

In [ ]:
output_directory = Path('output')

output_directory.mkdir(parents=True, exist_ok=True)

### Configurare stilizare

In [ ]:
# Creștem densitatea în pixeli a figurilor generate
rcParams['figure.dpi'] = 96

# Schimbm fontul default folosit pentru textul de pe grafice
rcParams['font.family'] = findfont(FontProperties(family=['sans-serif']))
rcParams['font.sans-serif'] = findfont(FontProperties(family=['IBM Plex Sans']))

# Set the default style to be used in making graphics
plt.style.use('default')

## Citire date

In [ ]:
bachelors_df = pd.read_excel(input_directory / 'CDOS Licență.xlsx', engine='openpyxl')
masters_df = pd.read_excel(input_directory / 'CDOS Master.xlsx', engine='openpyxl')

## Definire funcții de prelucrare date

In [ ]:
# Acest regex e folosit ca să elimine sufixul numeric
# adăugat automat de `pandas` pe coloanele cu nume duplicat.
#
# Ex. dacă coloana inițială se numea "test", puteam avea
# o coloană duplicată redenumită la "test.13".
remove_column_suffix_regex = re.compile(r'\.[0-9]+$')

def convert_column_name_to_question(column: str):
    """Transformă numele unei coloane într-o întrebare,
    așa cum apărea ea inițial în formular.
    """
    return remove_column_suffix_regex.sub('', column)

def strip_diacritics(s: str) -> str:
    """Transformă diacriticile dintr-un șir de caracatere
    în literele de bază.
    """
    # Bazat pe https://stackoverflow.com/a/518232/5723188
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def normalize(s: str) -> str:
    """Aduce șirurile de caractere la o formă mai ușor de comparat."""
    return strip_diacritics(s).lower()

## Generare grafice

In [ ]:
# Setare configurabilă pentru includerea întrebării în titlul graficelor

INCLUDE_QUESTION_TITLE = False

### Funcții comune

In [ ]:
def plot_question_title(ax: plt.Axes, answers: pd.Series) -> None:
    "Afișează întrebarea adresată respondenților ca titlul graficului."
    question = convert_column_name_to_question(answers.name)
    title = fill(question, 60)
    ax.set_title(title)

def plot_number_of_respondents(ax: plt.Axes, answers: pd.Series) -> None:
    "Afișează numărul de respondenți la o întrebare în partea de jos a graficului."
    ax.text(0, -1.3,
            f'{answers.count()} respondenți',
            ha='center', va='center')

def plot_to_file(plot_fn: Callable, answers: pd.Series, path: Path, **kwargs) -> None:
    "Afișează grafic răspunsurile la o întrebare și salvează ca fișier imaginea."
    fig = plt.figure()
    ax = fig.gca()
    plot_fn(ax, answers, **kwargs)
    fig.set_tight_layout(True)
    fig.savefig(path, bbox_inches='tight')
    plt.close(fig)
    
def col2index(col):
    """Convertește numele coloanei din Excel în indexul corespunzător."""
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num - 1

### Întrebări cu da sau nu

In [ ]:
NO_ANSWER_VALUE = 'Nu știu / nu răspund'

def plot_yes_no_question(ax: plt.Axes, answers: pd.Series) -> None:
    """Reprezintă grafic răspunsurile la o întrebare de tip da/nu,
    sub formă de pie chart.
    """
    # Înlocuiește răspunsurile goale cu o formulare standard
    answers = answers.fillna(NO_ANSWER_VALUE)
    
    # Calculează vectorul de frecvență al răspunsurilor
    counts = answers.value_counts()
    
    # Sortează tipurile de răspunsuri într-o ordine standard
    counts = counts.reindex(['Da', 'Nu', NO_ANSWER_VALUE], fill_value=0)
    
    # Elimin opțiunile care au 0 răspunsuri
    counts = counts[counts != 0]

    if INCLUDE_QUESTION_TITLE:
        plot_question_title(ax, answers)
    #plot_number_of_respondents(ax, answers)

    # Afișez vectorul de frecvență ca un pie chart
    counts.plot.pie(
        ax=ax,
        # pandas pune automat numele coloanei pe axa verticală,
        # nu ne dorim asta
        ylabel='',
        colors = ['#b62f8b','#d6556d','#fa9b3b'],
        autopct='%1.1f%%',
        normalize=True)

In [ ]:
answers = bachelors_df.iloc[:, col2index('NV')]

fig = plt.figure()
ax = fig.gca()
plot_yes_no_question(ax, answers)
plt.show()

### Întrebări cu rating de la 1 la 5

In [ ]:
def plot_rating_question(ax: plt.Axes, answers: pd.Series) -> None:
    answers = answers.dropna()
    
    # Dacă nu mai rămâne niciun răspuns, mă opresc
    if answers.empty:
        return
    
    # Calculează vectorul de frecvență al răspunsurilor
    counts = answers.value_counts()

    if counts.index.dtype == np.float32 or counts.index.dtype == np.float64:
        # Dintr-un motiv necunoscut, pandas interpretează numărul de răspunsuri ca float
        counts.index = counts.index.astype(int)

    counts = counts.sort_index()

    if INCLUDE_QUESTION_TITLE:
        plot_question_title(ax, answers)
    # TODO: nu arată bine
    #plot_number_of_respondents(ax, answers)
    counts.plot.bar(ax=ax, color = '#b62f8b')

    # Afișează vertical valorile pe axa X
    ax.tick_params(axis='x', rotation=0)

In [ ]:
answers = bachelors_df.iloc[:, col2index('KX')]

fig = plt.figure()
ax = fig.gca()
plot_rating_question(ax, answers)
plt.show()

### Întrebări cu răspuns unic

In [ ]:
def plot_single_choice_question(ax: plt.Axes, answers: pd.Series) -> None:
    # Ignor răspunsurile goale
    answers = answers.dropna()

    # Dacă nu mai rămâne nimic, continui
    if answers.empty:
        return

    # Determin frecvența cu care apare fiecare răspuns.
    counts = answers.value_counts()

    # Sortez după numărul de apariții
    counts = counts.sort_values()

    if INCLUDE_QUESTION_TITLE:
        plot_question_title(ax, answers)
    # TODO: nu se poziționează cum trebuie
    #plot_number_of_respondents(ax, answers)
    counts.plot.barh(ax=ax, color='#b62f8b')

In [ ]:
answers = bachelors_df.iloc[:, col2index('HF')]

fig = plt.figure()
ax = fig.gca()
plot_single_choice_question(ax, answers)
plt.show()

### Întrebări cu răspuns multiplu

In [ ]:
def plot_multiple_choice_question(ax: plt.Axes, answers: pd.Series, permitted_answers=None) -> None:
    # Ignor răspunsurile goale
    answers = answers.dropna()

    # Dacă nu mai rămâne nimic, nu plotez
    if answers.empty:
        return

    # Sparg coloana cu răspunsurile (un șir de alegeri separate prin virgulă)
    # în mai multe coloane.
    #
    # Bazat pe https://stackoverflow.com/a/46856366/5723188
    separated_answers = answers.str.split(', ', expand=True)
    
    # Determin frecvența cu care apare pe fiecare nouă coloană fiecare răspuns.
    #
    # Bazat pe https://stackoverflow.com/a/32589877/5723188
    counts = separated_answers.apply(pd.Series.value_counts)
    
    # Determin per total cât de frecvent a apărut fiecare răspuns la această întrebare.
    counts = counts.sum(axis=1)
    
    # Dacă ne-a fost transmisă o listă de valori permise după care trebuie să filtrăm
    if permitted_answers is not None:
        # Normalizez lista de intrare
        permitted_answers = [normalize(answer) for answer in permitted_answers]
        
        # Păstrăm doar valorile care se regăsesc în listă
        permitted_rows_mask = counts.index.map(normalize).isin(permitted_answers)
        counts = counts[permitted_rows_mask]
    
        # Dacă am filtrat totul, nu mai plotez nimic
        if counts.empty:
            return
    
    # Sortez după numărul de apariții
    counts = counts.sort_values()

    if INCLUDE_QUESTION_TITLE:
        plot_question_title(ax, answers)
    # TODO: nu se poziționează cum trebuie
    #plot_number_of_respondents(ax, answers, 4, -2.5)
    counts.plot.barh(ax=ax, color = '#b62f8b')

In [ ]:
answers = bachelors_df.iloc[:, col2index('HR')]

fig = plt.figure()
ax = fig.gca()
plot_multiple_choice_question(ax, answers)
plt.show()

## Licență - Întrebări pe specializări

### Mapare indici coloane

Studenții aleg inițial anul și specializarea lor. În funcție de acestea, sunt redirecționați la secțiuni diferite din formular.

Acest bloc de cod definește numărul coloanei de la care încep coloanele corespunzătoare fiecărei specializări.

Link util pentru conversia de la coloană Excel în indice: https://www.vishalon.net/blog/excel-column-letter-to-number-quick-reference

In [ ]:
# Dictionary mapping each (year, study program) pair to a column number in the Excel
bachelors_offset_map = {
    'I': {
        'Informatică': col2index('F'),
        'Informatică ID': col2index('BD'),
        'Matematică': col2index('CY'),
        'Calculatoare și Tehnologia Informației': col2index('HF'),
    },
    'II': {
        'Informatică': col2index('V'),
        'Matematici Aplicate': col2index('FZ'),
        'Matematică': col2index('DO'),
        'Matematică - Informatică': col2index('EU'),
        'Calculatoare și Tehnologia Informației': col2index('HU'),
    },
    'III': {
        'Informatică': col2index('AL'),
        'Matematici Aplicate': col2index('GO'),
        'Matematică': col2index('DO'),
        'Matematică - Informatică': col2index('FJ'),
        'Calculatoare și Tehnologia Informației': col2index('IJ'),
    },
    'IV': {
        'Calculatoare și Tehnologia Informației': col2index('IY'),
    }
}

### Listă de materii

Pentru a elimina răspunsurile invalide, definim listele de materii acceptabile pentru fiecare an și specializare.

In [ ]:
subjects_map = {
    'I': {
        'Informatică': [
            'Arhitectura sistemelor de calcul',
            'Baze de date',
            'Calcul diferențial și integral',
            'Competențe de bază într-o limbă străină',
            'Competențe specifice într-o limbă străină',
            'Gândire critică și etică academică în informatică',
            'Geometrie și algebră liniară',
            'Logică matematică și computațională',
            'Limbaje formale și automate',
            'Programarea algoritmilor',
            'Programare orientată pe obiecte',
            'Structuri algebrice în informatică',
            'Structuri de date',
            'Tehnici web'
        ],
        'Matematică': [
            'Algebra I',
            'Algebră II',
            'Algebra liniara',
            'Analiza Matematica I',
            'Analiză matematică II',
            'Competențe de bază într-o limbă străină',
            'Competențe specifice într-o limbă străină'
            'Elemente de calcul științific',
            'Gândire critică și etică academică',
            'Geometrie I',
            'Geometrie II',
            'Logică matematică',
            'Programare procedurală',
        ],
        'Calculatoare și Tehnologia Informației': [
            'Analiză matematică',
            'Arhitectura sistemelor de calcul',
            'Algebră și geometrie',
            'Bazele electrotehnicii',
            'Calcul numeric',
            'Competențe de bază într-o limbă străină',
            'Fizică',
            'Grafică asistată de calculator',
            'Matematici aplicate',
            'Proiectare logică',
            'Programarea calculatoarelor I',
            'Programarea calculatoarelor II',
            'Tehnici de programare',
            'Utilizarea sistemelor de operare',
        ],
    },
    'II': {
        'Informatică': [
            'Algoritmi fundamentali',
            'Dezvolarea aplicațiilor web',
            'Probabilități și statistică',
            'Programare funcțională',
            'Sisteme de gestiune a bazelor de date',
            'Sisteme de operare',
            'Competențe avansate într-o limbă straină',
            'Algoritmi avansați',
            'Fundamentele limbajelor de programare',
            'Inteligență artificială',
            'Metode de dezvoltare software',
            'Programare avansată pe obiecte',
            'Rețele de calculatoare'
        ],
        'Matematici Aplicate': [
            'Algebră 3',
            'Algebră 4',
            'Teoria măsurii și a integrării',
            'Geometrie diferențială',
            'Ecuații diferențiale',
            'Cercetări operaționale',
            'Criptografie',
            'Algoritmi fundamentali',
            'Probabilități',
            'Modelarea matematică a sistemelor materiale 1',
            'Analiză complexă',
            'Fizică',
            'Mecanică cerească',
            'Software matematic',
            'Inteligență artificială'
        ],
        'Matematică': [
            'Algebră 3',
            'Algebră 4',
            'Analiză complexă',
            'Analiză funcțională',
            'Ecuații diferențiale',
            'Geometrie diferențială 1',
            'Geometrie diferențială 2',
            'Modelarea matematică a sistemelor materiale 1',
            'Probabilități',
            'Teoria măsurii și a integrării',
            # La o întrebare este scris greșit numele disciplinei
            'Teoria măsurii și a interării',
            'Teoria numerelor',
        ],
        'Matematică - Informatică': [
            'Algebră 3',
            'Algebră 4',
            'Teoria măsurii și a integrării',
            'Geometrie diferențială 1',
            'Ecuații diferențiale',
            'Arhitectura sistemelor de calcul',
            'Sisteme de operare',
            'Programare orientată pe obiecte',
            'Analiză complexă',
            'Probabilități',
            'Tehnici Web',
            'Structuri de date',
            'Baze de date'
        ],
        'Calculatoare și Tehnologia Informației': [
            'Structuri de date și algoritmi',
            'Programare orientată pe obiecte',
            'Baze de date',
            'Fundamente ale rețelelor de calculatoare',
            'Elemente de electronică analogică',
            'Teoria sistemelor',
            'Competențe avansate într-o limbă străină I',
            'Probabilități și statistică',
            'Elemente avansate de programare',
            'Fundamente ale rutării în rețea',
            'Electronică digitală',
            'Calculatoare numerice',
            'Managementul și dezvoltarea carierei',
            'Competențe avansate într-o limbă străină II'
        ],
    },
    'III': {
        'Informatică': [
            'Calculabilitate și complexitate',
            'Securitatea sistemelor informatice',
            'Inginerie software',
            'Calcul numeric',
            'Concepte și aplicații în Vederea Artificială',
            'Dezvoltarea aplicațiilor în cloud prin intermediul platformei Salesforce',
            'Front end avansat: React & Angular',
            'Grafica pe calculator',
            'Introducere în programarea jocurilor pe calculator',
            'Introducere în Reinforcement Learning',
            'Introducere în robotica',
            'Java Script server side: Node.js + GraphQL',
            'Procesarea Semnalelor',
            'Robotic Process Automation (RPA) folosind platforma UiPath',
            'Tehnici de simulare',
            'Blockchain',
            'Elemente de securitate și logică aplicată',
            'Fundamentele proiectării compilatoarelor',
            'Introducere în prelucrarea limbajului natural',
            'Învățare automată în Vedere Artificială',
            'Învățarea rețelelor neurale adânci (Intensiv)',
            'Metode Formale în Ingineria Software',
            'Production engineering',
            'Programarea aplicațiilor de simulare',
            'Programarea dispozitivelor mobile în Android',
            'Programarea dispozitivelor mobile în iOS',
            'Protocoale criptografice',
            'Strategii de planificare a unei echipe de roboți',
            'Tehnici de programare a aplicațiilor grafice',
            'Testarea Sistemelor Software'
        ],
        'Matematici Aplicate': [
            'Statistică',
            'Analiză numerică',
            'Ecuații cu derivate parțiale',
            'Astronomie',
            'Mecanica mediilor continue'
        ],
        'Matematică': [
            'Analiză funcțională',
            'Analiză numerică',
            'Astronomie',
            'Complemente de geometrie',
            'Ecuații cu derivate parțiale',
            'Elemente de algebră modernă',
            'Geometrie diferențială',
            'Statistică',
        ],
        'Matematică - Informatică': [
            'Statistică',
            'Ecuații cu derivate parțiale',
            'Analiză numerică și metode numerice',
            'Cercetări operaționale',
            'Teoria numerelor cu aplicații în criptografie',
            'Front end avansat: React & Angular',
            'Java Script server side: Node.js + GraphQL',
            'Concepte și aplicații în Vederea Artificială',
            'Introducere în Reinforcement Learning',
            'Tehnici de simulare',
            'Grafica pe calculator',
            'Fundamentele proiectării compilatoarelor',
            'Metode Formale în Ingineria Software',
            'Programarea dispozitivelor mobile în iOS',
            'Strategii de planificare a unei echipe de roboți (SPER)',
            'Introducere în prelucrarea limbajului natural',
            'Programarea aplicațiilor de simulare',
            'Programarea dispozitivelor mobile în Android',
            'Protocoale criptografice',
            'Tehnici de programare a aplicațiilor grafice',
            'Blockchain',
            'Elemente de securitate și logică aplicată',
            'Învățare automată în Vedere Artificială',
            'Criptografie și teoria codurilor',
            'Elemente analiză clasică',
            'Grupuri de transformări în geometrie',
            'Grupuri Finite și Elemente de Teorie Galois',
            'Concepte Algebrice în Geometrie'
        ],
        'Calculatoare și Tehnologia Informației': [
            'Proiectarea bazelor de date',
            'Sisteme de operare',
            'Inteligenţă artificială',
            
        ],
    },
    'IV': {
        'Calculatoare și Tehnologia Informației': [
            'Sisteme distribuite',
            'Ingineria programării',
            'Dezvoltarea aplicațiilor WEB',
            'Comerț electronic',
            'Compilatoare și translatoare',
            'Testarea sistemelor software'
        ],
    }
}

### Generare grafice per an/specializare

In [ ]:
# Această expresie regulată e folosită ca să elimine seria
# din numele specializării.
remove_series_regex = re.compile(r' - seria [0-9]+$')

# Trebuie să luăm fiecare an de studiu pe rând
for index, year in enumerate(('I', 'II', 'III', 'IV')):
    # Filtrăm răspunsurile
    df_year = bachelors_df[bachelors_df['Anul de studiu în 2021-2022'] == f'Anul {year}'].copy()

    # Determinăm specializarea
    if year == 'IV':
        # Pentru anul 4 nu avem o coloană de "Specializare",
        # pentru că singura posibilitate ar fi fost CTI.
        #
        # Așa că o adăugăm manual.
        specialization_column_name = 'Specializare anul IV'
        df_year[specialization_column_name] = 'Calculatoare și Tehnologia Informației'
    else:
        # Coloana care indică specializarea e diferită în funcție de an
        specialization_column_index = 2 + index
        specialization_column_name = df_year.columns[specialization_column_index]

    specialization = df_year[specialization_column_name]
    specialization = specialization.str.replace(remove_series_regex, '')
    
    print('Anul', year, f'({len(df_year)} respondenți)')
    
    # Grupăm răspunsurile pe ani după specializare
    for group_name, df_group in df_year.groupby(specialization):
        # Sărim peste Info ID, de obicei sunt prea puține răspunsuri ca să fie relevante.
        if group_name == 'Informatică ID':
            continue
        
        print('-', group_name)
        offset = bachelors_offset_map[year][group_name]

        # Determin directorul
        directory_path = output_directory / f'Anul {year}' / group_name
        # Îl creez dacă nu există
        directory_path.mkdir(parents=True, exist_ok=True)
        
        # CTI are anumite caracteristici specifice
        is_cti = group_name.lower() == 'Calculatoare și Tehnologia Informației'.lower()
        is_info_year_3 = (year == 'III') and (group_name.lower() == 'Informatică'.lower())
        
        if is_cti or is_info_year_3:
            # La CTI ordinea întrebărilor este puțin diferită
            multiple_choice_question_indices = [0, 2, 4, 5, 9, 10]
            rating_question_indices = [13]
            yes_no_question_indices = [7, 12]
        else:
            multiple_choice_question_indices = [0, 2, 4, 5, 9, 10]
            rating_question_indices = [13]
            yes_no_question_indices = [12]
    
        # Cei din an terminal au o întrebare legată și de master
        if (year == 'III' and not is_cti) or year == 'IV':
            yes_no_question_indices.append(16)

        # Rezolv întrebările de tip selecție multiplă
        for i in multiple_choice_question_indices:
            index = offset + i

            answers = df_group.iloc[:, index]
            
            permitted_answers = None
            
            # Prima întrebare are ca opțiuni intervale de procente, nu materii
            if i != 0:
                # Obțin lista de materii pentru această specializare
                known_subjects = subjects_map[year][group_name]
                permitted_answers = known_subjects

            # Salvez graficul într-un fișier
            plot_to_file(plot_multiple_choice_question, answers, directory_path / f'{i}.png',
                         permitted_answers=permitted_answers)
        
        # Întrebările de tip rating
        for i in rating_question_indices:
            index = offset + i

            answers = df_group.iloc[:, index]

            plot_to_file(plot_rating_question, answers, directory_path / f'{i}.png')

        # Întrebările de tip da/nu/nu știu
        for i in yes_no_question_indices:
            index = offset + i

            answers = df_group.iloc[:, index]

            plot_to_file(plot_yes_no_question, answers, directory_path / f'{i}.png')

## Master - Întrebări pe specializări

In [ ]:
# Dictionary mapping each (year, study program) pair to a column number in the Excel
masters_offset_map = {
    'I': {
        'ARTIFICIAL INTELLIGENCE': col2index('EH'),
        'MATEMATICĂ DIDACTICĂ': col2index('IL'),
        'PROBABILITĂȚI ȘI STATISTICĂ ÎN FINANȚE ȘI ȘTIINȚE': col2index('JN'),
        'BAZE DE DATE ȘI TEHNOLOGII SOFTWARE': col2index('CF'),
        'INGINERIE SOFTWARE': col2index('DG'),
    },
    'II': {
        'Baze de date și tehnologii software': col2index('BS'),
        'Securitate și Logică Aplicată': col2index('S')
    },
}

In [ ]:
# Trebuie să luăm anii de studiu pe rând
for index, year in enumerate(('I', 'II')):
    # Filtrăm răspunsurile
    df_year = masters_df[masters_df['Anul de studiu în 2020-2021'] == f'Anul {year}'].copy()

    # Coloana care indică specializarea e diferită în funcție de an
    specialization_column_index = 2 + index
    specialization_column_name = df_year.columns[specialization_column_index]

    specialization = df_year[specialization_column_name]

    print('Anul', year, f'({len(df_year)} respondenți)')

    # Grupăm răspunsurile pe ani după specializare
    for group_name, df_group in df_year.groupby(specialization):
        print('-', group_name)
        offset = masters_offset_map[year][group_name]

        # Determin directorul
        directory_path = output_directory / f'Master - Anul {year}' / group_name
        # Îl creez dacă nu există
        directory_path.mkdir(parents=True, exist_ok=True)
        
        if year == 'II':
            multiple_choice_question_indices = list(range(3)) + list(range(4, 10))
            rating_question_indices = [10]
            yes_no_question_indices = [11, 12]
        else:
            multiple_choice_question_indices = list(range(3)) + list(range(5, 11))
            rating_question_indices = [11]
            yes_no_question_indices = [12, 13]
    
        # Rezolv întrebările de tip selecție multiplă
        for i in multiple_choice_question_indices:
            index = offset + i

            answers = df_group.iloc[:, index]

            # Salvez graficul într-un fișier
            plot_to_file(plot_multiple_choice_question, answers, directory_path / f'{i}.png')
        
        # Întrebările de tip rating
        for i in rating_question_indices:
            index = offset + i

            answers = df_group.iloc[:, index]

            plot_to_file(plot_rating_question, answers, directory_path / f'{i}.png')

        # Întrebările de tip da/nu/nu știu
        for i in yes_no_question_indices:
            index = offset + i

            answers = df_group.iloc[:, index]

            plot_to_file(plot_yes_no_question, answers, directory_path / f'{i}.png')

## Întrebări comune

In [ ]:
common_directory = output_directory / 'Comune'
common_directory.mkdir(parents=True, exist_ok=True)

### Configurație întrebări

Aici definim secțiunile în care se încadrează întrebările și tipul lor.

In [ ]:
# Constante pentru tipurile de întrebări
SINGLE_CHOICE = 'single_choice'
MULTIPLE_CHOICE = 'multiple_choice'
YES_NO = 'yes_no'
RATING = 'rating'

# Un dicționar care împarte întrebările pe secțiuni, și definește pentru fiecare secțiune 
common_questions = {
    'Desfășurarea orelor în format online': {
        SINGLE_CHOICE: ['HE', 'HK', 'HL', 'HM', 'HN', 'HO', 'HT', 'HU',],
        MULTIPLE_CHOICE: ['HD', 'HQ', 'HR',],
        YES_NO: ['HF', 'HP', 'HS',],
        RATING: ['HG', 'HH', 'HI',],
     },
    'Bibliotecă': {
        YES_NO: ['HY', 'HZ', 'IA', 'IB', 'IC', 'ID', 'IF',],
    },
    'Site & Moodle': {
        SINGLE_CHOICE: [ 'IJ', 'IK', 'IL',],
        RATING: ['IG', 'II',],
    },
    'Modulul psihopedagogic': {
        SINGLE_CHOICE: ['IN',],
        RATING: ['IO',],
    },
    'Reprezentarea studenților': {
        SINGLE_CHOICE: ['IX', 'IY',],
        YES_NO: ['JA', ],
        RATING: ['JB', ],
    },
    'Secretariat și orar': {
        SINGLE_CHOICE: ['JI', 'JK', 'JM', 'JN', ],
        RATING: ['JE', 'JH', 'JJ',],
    },
    'Carieră': {
        SINGLE_CHOICE: ['JS',],
        MULTIPLE_CHOICE: ['JQ',],
        YES_NO: ['JP',],
    },
    'Cămine': {
        SINGLE_CHOICE: ['JW',],
        YES_NO: ['JV',],
        RATING: ['JX', 'JY', 'JZ', 'KA', 'KB', 'KC', 'KD', 'KE', 'KF', 'KG',],
    },
    'Cantină': {
        YES_NO: ['KI',],
        RATING: ['KJ', 'KK', 'KL',],
    },
    'Sport': {
        SINGLE_CHOICE: ['KO', 'KP',]
    },
    'Conducerea Facultății': {
        YES_NO: ['KU', 'KY',],
    },
    'ASMI': {
        SINGLE_CHOICE: ['LQ', 'LR',],
        YES_NO: ['LL', 'LM', 'LS',],
        RATING: ['LA', 'LB', 'LC', 'LD', 'LE', 'LF', 'LG', 'LH', 'LN', 'LP',],
    },
    'Practică': {
        YES_NO: ['LT',],
        RATING: ['LW', 'LZ', 'MB', 'MC', 'MD',],
    },
    'Departamentul de Consiliere și Orientare pentru Carieră': {
        YES_NO: ['ME', 'MF', 'MH', ]
    },
    'Șefi de grupă': {
        YES_NO: ['MI', 'MJ', 'MK', 'ML',]
    },
    'Activități extra': {
        MULTIPLE_CHOICE: ['MM',]
    },
}

### Generare grafice comune

Acest cod parcurge secțiunile definite mai sus și generează grafice pentru întrebările din fiecare.

In [ ]:
# Anul acesta le-am pus la comun cu răspunsurile de la licență
common_df = bachelors_df

for section, question_types in common_questions.items():
    section_output_directory = common_directory / section
    section_output_directory.mkdir(parents=True, exist_ok=True)

    def get_question_indices_by_type(question_type):
        columns = question_types.get(question_type, [])
        return map(col2index, columns)
    
    def plot_answers(plot_fn, indices):
        for index in indices:
            answers = common_df.iloc[:, index]
            
            # Pentru secțiunea „Bibliotecă”, vrem să ignorăm respondenții
            # care au zis că nu au permis de bibliotecă
            if section == 'Bibliotecă' and index >= col2index('IA'):
                filter_column_index = col2index('HZ')
                has_answer = common_df.iloc[:, filter_column_index] == 'Da'
                answers = answers[has_answer]
            
            plot_to_file(plot_fn, answers, section_output_directory / f'{index}.png')

    print(section)

    # Extragem din dicționar întrebările de tip selecție unică
    single_choice_question_indices = get_question_indices_by_type(SINGLE_CHOICE)
    plot_answers(plot_single_choice_question, single_choice_question_indices)

    # Extragem din dicționar întrebările de tip selecție multiplă
    multiple_choice_question_indices = get_question_indices_by_type(MULTIPLE_CHOICE)
    plot_answers(plot_multiple_choice_question, multiple_choice_question_indices)
    
    # Extragem din dicționar întrebările de tip da/nu
    yes_no_question_indices = get_question_indices_by_type(YES_NO)
    plot_answers(plot_yes_no_question, yes_no_question_indices)
    
    # Extragem din dicționar întrebările de tip rating
    rating_question_indices = get_question_indices_by_type(RATING)
    plot_answers(plot_rating_question, rating_question_indices)